<a href="https://colab.research.google.com/github/Dagobert42/langID-NLP/blob/main/langID_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import string
import re
from nltk import ngrams
from collections import defaultdict
import nltk
import collections
import torch

In [6]:
# read data
# this was written for the WiLI-2018 data set: https://zenodo.org/record/841984
# make sure txt-files are in the specified directory when running this 
X_train = open('x_train.txt', encoding="utf8").read().split('\n')
Y_train = open('y_train.txt', encoding="utf8").read().split('\n')
labels = pd.read_csv('labels.csv', delimiter = ';')
Y_train =  Y_train[:-1]
X_train =  X_train[:-1]

In [7]:
# remove unnecessary characters from data
extras = '!"$%&/{}[]()=?\\`´*+~#-_.:,;<>|1234567890°-\'' # Characters to remove from data
rx = '[' + re.escape(''.join(extras)) + ']'
x_train = [] 
for example in X_train:
    x_train.append(re.sub(' +', ' ', re.sub(rx, '', example)))

In [8]:
# convert language labels to language Name => 'en' -> 'English'
lab_dict = { labels.loc[i]['Label'] : labels.loc[i]['English'] for i in range(0, len(labels)) }
y_train = [ lab_dict[item] if item != 'nan' else 'Min Nan Chinese' for item in Y_train ]

In [9]:
# collecting examples belonging to same language
# lang_2_ex = { 'English' : [ 'Welcome to ..', 'PHP sucks ...', ...] , 'Spanish' : ['dads....', 'dasdsa...',....] , ....}
lang_2_ex = defaultdict(list)
for i in range(len(x_train)):
    lang_2_ex[y_train[i]].append(x_train[i])

In [10]:
# creating n-grams for each language 
n = 3 # beginning with trigrams
gram_per_lang = defaultdict(list)
uniq_lang = set(y_train)
for lang in uniq_lang:
    for sent in lang_2_ex[lang]:
        gram_per_lang[lang] += [sent[i : i+n] for i in range(len(sent)-n+1)]

In [11]:
# calculating term frequency of n-grams per language

tf_per_lang = defaultdict(list)
for lang in uniq_lang:
    tf_per_lang[lang] = dict(zip(list(collections.Counter(gram_per_lang[lang]).keys()), list(collections.Counter(gram_per_lang[lang]).values())))

In [12]:
# sort term frequency per language 

sorted_tf_per_lang = defaultdict(list)
for lang in uniq_lang:
    sorted_tf_per_lang[lang] = {word : value for word, value in sorted(tf_per_lang[lang].items(), key=lambda item : item[1], reverse=True)}

In [19]:
# print some examples
languages = ['German', 'English', 'Arabic', 'Japanese', 'Armenian']
n_examples = 10
for lang_key in languages:
    print(lang_key)
    print(list(sorted_tf_per_lang[lang_key].keys())[:n_examples])
    print(list(sorted_tf_per_lang[lang_key].values())[:n_examples])

German
['en ', 'er ', ' de', 'der', 'sch', 'ie ', 'che', 'nd ', 'ein', 'ch ']
[3915, 3021, 2241, 1655, 1473, 1336, 1184, 1175, 1101, 1073]
English
[' th', 'he ', 'the', 'ed ', ' in', ' an', 'nd ', 'and', ' of', 'of ']
[2838, 2765, 2569, 1660, 1371, 1326, 1287, 1277, 1240, 1182]
Arabic
[' ال', 'الم', 'ية ', 'في ', ' في', 'ة ا', ' من', 'من ', 'ن ا', 'ات ']
[8365, 1595, 1468, 1441, 1424, 1378, 1168, 1086, 1084, 937]
Japanese
['ている', 'いる。', 'である', 'ある。', 'された', 'った。', 'として', 'れてい', 'してい', 'ていた']
[344, 263, 256, 237, 231, 226, 220, 186, 185, 182]
Armenian
['ում', 'ան ', 'ակա', 'ներ', 'ւմ ', 'կան', 'ին ', 'ութ', 'ուն', 'ւթյ']
[2006, 1889, 1633, 1531, 1516, 1467, 1383, 1345, 1331, 1207]
